In [ ]:
import numpy as np
import time
import pybullet as p
import argparse
from gym_pybullet_drones.utils.utils import sync
from gym_pybullet_drones.envs.BaseAviary import DroneModel, Physics
from gym_pybullet_drones.envs.CtrlAviary import CtrlAviary
from gym_pybullet_drones.control.DSLPIDControl import DSLPIDControl
from gym_pybullet_drones.utils.Logger import Logger

In [ ]:
class DroneSwarmSimulation:
    def __init__(self, 
                 num_drones=15,
                 simulation_freq_hz=240,
                 control_freq_hz=48,
                 duration_sec=60,
                 gui=True,
                 record=False,
                 time_step=1/240,
                 max_steps=6000):
        
        # Disconnect any existing PyBullet connections
        try:
            if p.isConnected():
                p.disconnect()
        except:
            pass
        
        # Initialize simulation parameters
        self.NUM_DRONES = num_drones
        self.SIMULATION_FREQ_HZ = simulation_freq_hz
        self.CONTROL_FREQ_HZ = control_freq_hz
        self.DURATION_SEC = duration_sec
        self.GUI = gui
        self.RECORD = record
        self.time_step = time_step
        self.max_steps = max_steps
        self.avoidance_strength = 5.0  # How strongly drones avoid each other
        
        # Initialize the simulation
        self.setup_simulation()

In [ ]:
def setup_simulation(self):
        """Set up the simulation environment and drones"""
        # Initial drone positions in a grid pattern
        initial_xyzs = np.zeros((self.NUM_DRONES, 3))
        rows = int(np.sqrt(self.NUM_DRONES))
        cols = int(np.ceil(self.NUM_DRONES / rows))
        
        for i in range(self.NUM_DRONES):
            row, col = i // cols, i % cols
            initial_xyzs[i] = [(col - cols/2 + 0.5) * 0.5, 
                              (row - rows/2 + 0.5) * 0.5, 
                              0.5]
        
        # Explicitly connect to PyBullet before creating the environment
        if self.GUI:
            self.client = p.connect(p.GUI)
            print(f"Connected to PyBullet with GUI mode, client ID: {self.client}")
        else:
            self.client = p.connect(p.DIRECT)
            print(f"Connected to PyBullet with DIRECT mode, client ID: {self.client}")
        
        # Initialize the environment
        self.env = CtrlAviary(
            drone_model=DroneModel.CF2X,
            num_drones=self.NUM_DRONES,
            initial_xyzs=initial_xyzs,
            initial_rpys=np.zeros((self.NUM_DRONES, 3)),
            physics=Physics.PYB,
            gui=self.GUI,
            record=self.RECORD,
            obstacles=False,
            user_debug_gui=True
        )
        
        # Initialize controllers for each drone
        self.ctrl = [DSLPIDControl(drone_model=DroneModel.CF2X) for _ in range(self.NUM_DRONES)]
        
        # Initialize target positions and orientations
        self.INIT_XYZS = np.array([self.env.INIT_XYZS[i, :] for i in range(self.NUM_DRONES)])
        self.TARGET_XYZ = self.INIT_XYZS.copy()
        self.INIT_RPYS = np.array([self.env.INIT_RPYS[i, :] for i in range(self.NUM_DRONES)])
        self.TARGET_RPYS = np.zeros((self.NUM_DRONES, 3))
        
        # Initialize action array
        self.action = np.zeros((self.NUM_DRONES, 4))
        
        # Initialize logger
        self.logger = Logger(logging_freq_hz=self.CONTROL_FREQ_HZ,
                             num_drones=self.NUM_DRONES,
                             duration_sec=self.DURATION_SEC)
        
        # Formation state variables
        self.current_formation = "takeoff"
        self.formation_change_time = 10  # Change formation every X seconds
        self.last_formation_change = 0
        self.available_formations = ["circle", "grid", "pyramid", "random", "line"]

In [ ]:
def set_circle_formation(self, height=1.5, radius=3.0):
        """Set target positions for a circle formation"""
        self.TARGET_XYZ = np.array([
                [np.cos(i/self.NUM_DRONES*2*np.pi)*radius, 
                 np.sin(i/self.NUM_DRONES*2*np.pi)*radius, 
                 height] 
                for i in range(self.NUM_DRONES)
            ])
        # Set drones to face center
        for i in range(self.NUM_DRONES):
            # Calculate angle to face center
            angle = np.arctan2(-self.TARGET_XYZ[i, 1], -self.TARGET_XYZ[i, 0])
            self.TARGET_RPYS[i] = [0, 0, angle]


In [ ]:
def set_grid_formation(self, height=1.5, spacing=0.8):
        """Set target positions for a grid formation"""
        rows = int(np.sqrt(self.NUM_DRONES))
        cols = int(np.ceil(self.NUM_DRONES / rows))
        
        self.TARGET_XYZ = np.zeros((self.NUM_DRONES, 3))
        for i in range(self.NUM_DRONES):
            row, col = i // cols, i % cols
            self.TARGET_XYZ[i] = [(col - cols/2 + 0.5) * spacing, 
                                 (row - rows/2 + 0.5) * spacing, 
                                 height]
            

In [ ]:
 def set_pyramid_formation(self, base_height=0.5, spacing=0.8):
        """Set target positions for a pyramid formation"""
        layer = 0
        drones_in_layer = 1
        drones_placed = 0
        
        self.TARGET_XYZ = np.zeros((self.NUM_DRONES, 3))
        

In [ ]:
def set_pyramid_formation(self, base_height=0.5, spacing=0.8):
        """Set target positions for a pyramid formation"""
        layer = 0
        drones_in_layer = 1
        drones_placed = 0
        
        self.TARGET_XYZ = np.zeros((self.NUM_DRONES, 3))
        
        while drones_placed < self.NUM_DRONES:
            layer_size = min(drones_in_layer**2, self.NUM_DRONES - drones_placed)
            layer_rows = int(np.sqrt(layer_size))
            layer_cols = int(np.ceil(layer_size / layer_rows))
            
            for i in range(layer_size):
                if drones_placed >= self.NUM_DRONES:
                    break
                    
                row, col = i // layer_cols, i % layer_cols
                self.TARGET_XYZ[drones_placed] = [
                    (col - layer_cols/2 + 0.5) * spacing,
                    (row - layer_rows/2 + 0.5) * spacing,
                    base_height + layer * spacing
                ]
                drones_placed += 1
                
            layer += 1
            drones_in_layer += 1


In [ ]:
def set_random_formation(self, min_height=1.0, max_height=2.0, area_size=4.0):
        """Set random target positions with collision avoidance"""
        self.TARGET_XYZ = np.array([
            [np.random.uniform(-area_size/2, area_size/2), 
             np.random.uniform(-area_size/2, area_size/2),
             np.random.uniform(min_height, max_height)]
            for _ in range(self.NUM_DRONES)
        ])
        
        # Simple collision avoidance for target positions
        min_distance = 0.5
        for _ in range(5):  # Multiple iterations to resolve conflicts
            for i in range(self.NUM_DRONES):
                for j in range(i+1, self.NUM_DRONES):
                    distance = np.linalg.norm(self.TARGET_XYZ[i, :2] - self.TARGET_XYZ[j, :2])
                    if distance < min_distance:
                        # Move drones away from each other
                        direction = self.TARGET_XYZ[i, :2] - self.TARGET_XYZ[j, :2]
                        if np.linalg.norm(direction) > 0:
                            direction = direction / np.linalg.norm(direction)
                        else:
                            direction = np.random.rand(2) * 2 - 1  # Random direction if overlapping
                            direction = direction / np.linalg.norm(direction)
                        
                        push_distance = (min_distance - distance) / 2
                        self.TARGET_XYZ[i, :2] += direction * push_distance
                        self.TARGET_XYZ[j, :2] -= direction * push_distance
                        
                        # Keep within bounds
                        self.TARGET_XYZ[i, :2] = np.clip(self.TARGET_XYZ[i, :2], -area_size/2, area_size/2)
                        self.TARGET_XYZ[j, :2] = np.clip(self.TARGET_XYZ[j, :2], -area_size/2, area_size/2)


In [ ]:
def set_line_formation(self, height=1.5, spacing=0.5):
        """Set target positions for a line formation"""
        self.TARGET_XYZ = np.zeros((self.NUM_DRONES, 3))
        for i in range(self.NUM_DRONES):
            offset = (i - (self.NUM_DRONES-1)/2) * spacing
            self.TARGET_XYZ[i] = [offset, 0, height]

In [ ]:
 def update_formation(self, current_time):
        """Update formation based on time"""
        if current_time - self.last_formation_change > self.formation_change_time:
            # Choose next formation
            formations = list(self.available_formations)
            if self.current_formation in formations:
                formations.remove(self.current_formation)
            self.current_formation = np.random.choice(formations)
            
            # Set the new formation
            if self.current_formation == "circle":
                self.set_circle_formation()
            elif self.current_formation == "grid":
                self.set_grid_formation()
            elif self.current_formation == "pyramid":
                self.set_pyramid_formation()
            elif self.current_formation == "random":
                self.set_random_formation()
            elif self.current_formation == "line":
                self.set_line_formation()
            
            # Update formation change time
            self.last_formation_change = current_time
            print(f"Changed to {self.current_formation} formation at time {current_time:.1f}s")

In [ ]:
 def control_drones(self, i):
        """Calculate control inputs for each drone"""
        # Calculate current simulation time
        current_time = i / self.SIMULATION_FREQ_HZ
        
        # Start with takeoff for the first 10 seconds
        if current_time < 10:
            if self.current_formation != "takeoff":
                print("Starting takeoff phase...")
                self.current_formation = "takeoff"
                
            # Staggered takeoff
            for j in range(self.NUM_DRONES):
                takeoff_delay = j * 0.5  # 0.5 second delay between each drone
                if current_time > takeoff_delay:
                    takeoff_height = min(1.5, (current_time - takeoff_delay) * 0.3)
                    takeoff_pos = self.INIT_XYZS[j].copy()
                    takeoff_pos[2] = takeoff_height
                    
                    self.action[j, :] = self.ctrl[j].computeControlFromState(
                        control_timestep=self.SIMULATION_FREQ_HZ/self.CONTROL_FREQ_HZ,
                        state=self.env._getDroneStateVector(j),
                        target_pos=takeoff_pos,
                        target_rpy=self.TARGET_RPYS[j]
                    )
        else:
            # Normal operation after takeoff
            # Update formation if needed
            self.update_formation(current_time)
            
            # Control each drone
            for j in range(self.NUM_DRONES):
                state = self.env._getDroneStateVector(j)
                
                # Add some subtle hovering motion
                hover_offset = np.sin(current_time + j * 0.5) * 0.05
                target_pos = self.TARGET_XYZ[j].copy()
                target_pos[2] += hover_offset
                
                self.action[j, :] = self.ctrl[j].computeControlFromState(
                    control_timestep=self.SIMULATION_FREQ_HZ/self.CONTROL_FREQ_HZ,
                    state=state,
                    target_pos=target_pos,
                    target_rpy=self.TARGET_RPYS[j]
                )

In [ ]:
 def run_simulation(self):
        """Run the complete simulation"""
        print(f"Starting simulation with {self.NUM_DRONES} drones...")
        START = time.time()
        
        # Main simulation loop
        for i in range(0, int(self.DURATION_SEC*self.SIMULATION_FREQ_HZ)):
            # Step the simulation
            if i % (self.SIMULATION_FREQ_HZ//self.CONTROL_FREQ_HZ) == 0:
                self.control_drones(i)
            
            self.env.step(self.action)
            
            # Log data for drone 0 (as an example)
            if i % (self.SIMULATION_FREQ_HZ//self.CONTROL_FREQ_HZ) == 0:
                self.logger.log(
                    drone=0,
                    timestamp=i/self.SIMULATION_FREQ_HZ,
                    state=self.env._getDroneStateVector(0),
                    control=np.zeros(12)  # Simplified control logging
                )
            
            # Sync the simulation with real time if GUI is enabled
            if self.GUI:
                sync(i, START, self.SIMULATION_FREQ_HZ)
        
        # Close the environment
        self.env.close()
        
        # Plot results
        self.logger.plot()

In [ ]:
def __init__(self, 
             num_drones=15,
             simulation_freq_hz=240,
             control_freq_hz=48,
             duration_sec=60,
             gui=True,
             record=False):
    
    # Disconnect any existing PyBullet connections
    try:
        if p.isConnected():
            p.disconnect()
    except:
        pass
    
    # Initialize simulation parameters
    self.NUM_DRONES = num_drones
    self.SIMULATION_FREQ_HZ = simulation_freq_hz
    self.CONTROL_FREQ_HZ = control_freq_hz
    self.DURATION_SEC = duration_sec
    self.GUI = gui
    self.RECORD = record
    
    # Explicitly connect to PyBullet before creating the environment
    if self.GUI:
        self.client = p.connect(p.GUI)
        print(f"Connected to PyBullet with GUI mode, client ID: {self.client}")
    else:
        self.client = p.connect(p.DIRECT)
        print(f"Connected to PyBullet with DIRECT mode, client ID: {self.client}")

In [ ]:
#without reward system

import pybullet as p
import pybullet_data
import time
import numpy as np
import os

# Disconnect any existing connections
try:
    if p.isConnected():
        p.disconnect()
except:
    pass

# Try to connect with GUI
client = p.connect(p.GUI)
print(f"Connected with client ID: {client}")

# Set the path to PyBullet's data
p.setAdditionalSearchPath(pybullet_data.getDataPath())
print(f"Search path set to: {pybullet_data.getDataPath()}")

# Load floor plane
plane_id = p.loadURDF("plane.urdf")

# Set gravity
p.setGravity(0, 0, -9.8)

# Configure camera view
p.resetDebugVisualizerCamera(
    cameraDistance=10,
    cameraYaw=45,
    cameraPitch=-30,
    cameraTargetPosition=[0, 0, 2]
)

# Number of drones
num_drones = 10

# Function to create a drone with rotors
def create_drone(position, color=None):
    if color is None:
        color = [0.7, 0.7, 0.7, 1]
    
    # Create the main body of the drone
    body_radius = 0.1
    body_height = 0.05
    body_visual = p.createVisualShape(
        shapeType=p.GEOM_CYLINDER,
        radius=body_radius,
        length=body_height,
        rgbaColor=color
    )
    body_collision = p.createCollisionShape(
        shapeType=p.GEOM_CYLINDER,
        radius=body_radius,
        height=body_height
    )
    
    # Create the drone base body
    drone_id = p.createMultiBody(
        baseMass=0.3,
        baseCollisionShapeIndex=body_collision,
        baseVisualShapeIndex=body_visual,
        basePosition=position,
        baseOrientation=p.getQuaternionFromEuler([0, 0, 0])
    )
    
    # Add four arms
    arm_length = 0.15
    arm_width = 0.02
    arm_height = 0.01
    arm_positions = [
        [arm_length/2, 0, 0],
        [0, arm_length/2, 0],
        [-arm_length/2, 0, 0],
        [0, -arm_length/2, 0]
    ]
    arm_orientations = [
        p.getQuaternionFromEuler([0, 0, 0]),
        p.getQuaternionFromEuler([0, 0, np.pi/2]),
        p.getQuaternionFromEuler([0, 0, 0]),
        p.getQuaternionFromEuler([0, 0, np.pi/2])
    ]
    
    for i, (arm_pos, arm_orn) in enumerate(zip(arm_positions, arm_orientations)):
        arm_visual = p.createVisualShape(
            shapeType=p.GEOM_BOX,
            halfExtents=[arm_length/2, arm_width/2, arm_height/2],
            rgbaColor=color
        )
        arm_collision = p.createCollisionShape(
            shapeType=p.GEOM_BOX,
            halfExtents=[arm_length/2, arm_width/2, arm_height/2]
        )
        
        p.createMultiBody(
            baseMass=0.05,
            baseCollisionShapeIndex=arm_collision,
            baseVisualShapeIndex=arm_visual,
            basePosition=[position[0] + arm_pos[0], position[1] + arm_pos[1], position[2] + arm_pos[2]],
            baseOrientation=arm_orn,
            baseInertialFramePosition=[0, 0, 0],
            baseInertialFrameOrientation=[0, 0, 0, 1],
            linkMasses=[],
            linkCollisionShapeIndices=[],
            linkVisualShapeIndices=[],
            linkPositions=[],
            linkOrientations=[],
            linkInertialFramePositions=[],
            linkInertialFrameOrientations=[],
            linkParentIndices=[],
            linkJointTypes=[],
            linkJointAxis=[]
        )
    
    # Add rotors at the end of each arm
    rotor_radius = 0.05
    rotor_height = 0.01
    rotor_positions = [
        [arm_length, 0, arm_height],
        [0, arm_length, arm_height],
        [-arm_length, 0, arm_height],
        [0, -arm_length, arm_height]
    ]
    
    rotor_ids = []
    for rotor_pos in rotor_positions:
        rotor_visual = p.createVisualShape(
            shapeType=p.GEOM_CYLINDER,
            radius=rotor_radius,
            length=rotor_height,
            rgbaColor=[0.2, 0.2, 0.2, 0.8]
        )
        rotor_collision = p.createCollisionShape(
            shapeType=p.GEOM_CYLINDER,
            radius=rotor_radius,
            height=rotor_height
        )
        
        rotor_id = p.createMultiBody(
            baseMass=0.02,
            baseCollisionShapeIndex=rotor_collision,
            baseVisualShapeIndex=rotor_visual,
            basePosition=[position[0] + rotor_pos[0], position[1] + rotor_pos[1], position[2] + rotor_pos[2]],
            baseOrientation=p.getQuaternionFromEuler([0, 0, 0])
        )
        rotor_ids.append(rotor_id)
    
    # Create a constraint to connect all parts together
    # In a more advanced setup, you'd create proper joints
    # For simplicity, we'll just track these separately and move them together
    
    return drone_id, rotor_ids

# Create drones with all their components
drone_systems = []
for i in range(num_drones):
    # Calculate position in a circle
    angle = 2 * np.pi * i / num_drones
    x = 3 * np.cos(angle)
    y = 3 * np.sin(angle)
    z = 1.5  # Start at same height
    
    # Create a colored drone
    color = [i/num_drones, 0.5, 1-i/num_drones, 1]  # Different color for each drone
    drone_id, rotor_ids = create_drone([x, y, z], color)
    drone_systems.append((drone_id, rotor_ids))

# Various formation patterns
def circle_formation(t, i, num_drones):
    radius = 3 + np.sin(t/2) * 1.5  # Dynamic radius
    angle = 2 * np.pi * i / num_drones + t / 5  # Rotating circle
    x = radius * np.cos(angle)
    y = radius * np.sin(angle)
    z = 1.5 + np.sin(t/3 + i * 0.5) * 0.5  # Vary height
    # Orientation: face the direction of travel
    orientation = np.arctan2(-np.sin(angle + t/5), -np.cos(angle + t/5))
    return [x, y, z, orientation]

def figure_eight_formation(t, i, num_drones):
    # Phase difference between drones
    phase = i * 2 * np.pi / num_drones
    # Figure-8 parametric equation
    x = 4 * np.sin(t/2 + phase)
    y = 4 * np.sin(t/2 + phase) * np.cos(t/2 + phase)
    z = 1.5 + np.sin(t + phase) * 0.5
    # Calculate direction of travel for orientation
    dx = 4 * np.cos(t/2 + phase)/2
    dy = 4 * (np.cos(t/2 + phase) * np.cos(t/2 + phase) - np.sin(t/2 + phase) * np.sin(t/2 + phase))/2
    orientation = np.arctan2(dy, dx)
    return [x, y, z, orientation]

def spiral_formation(t, i, num_drones):
    # Spiral with increasing radius
    angle = 4 * t + i * 2 * np.pi / num_drones
    radius = 1 + (i / num_drones) * 3
    x = radius * np.cos(angle)
    y = radius * np.sin(angle)
    z = 1.5 + (i / num_drones) * 2
    # Face the center of the spiral
    orientation = np.arctan2(-y, -x)
    return [x, y, z, orientation]

def wave_formation(t, i, num_drones):
    # Line with sine wave
    x = (i - num_drones/2) * 0.8
    y = np.sin(t + i * 0.5) * 2
    z = 1.5 + np.cos(t/2 + i * 0.3) * 0.7
    # Face the direction of travel
    dy = np.cos(t + i * 0.5) * 2
    orientation = np.arctan2(dy, 0)
    return [x, y, z, orientation]

# List of formations
formations = [
    circle_formation,
    figure_eight_formation,
    spiral_formation,
    wave_formation
]

# Run simulation
print("Running simulation with dynamic drone movements...")
current_formation = 0
formation_change_time = 500  # Steps before changing formation
formation_transition = 100   # Steps for smooth transition

# PD controller parameters for smoother movement
kp = 10.0  # Position gain
kd = 5.0   # Velocity gain

# Target positions and velocities
target_positions = [np.zeros(4) for _ in range(num_drones)]  # x, y, z, orientation
current_velocities = [np.zeros(3) for _ in range(num_drones)]

# Rotor animation speed
rotor_speed = 30.0  # radians per second
rotor_angles = np.zeros((num_drones, 4))  # Track rotation angles for each rotor

for step in range(6000):
    # Change formation periodically
    if step % formation_change_time == 0:
        prev_formation = current_formation
        current_formation = (current_formation + 1) % len(formations)
        print(f"Changing to formation: {formations[current_formation].__name__}")
    
    # Calculate blend factor for smooth transition
    in_transition = step % formation_change_time < formation_transition
    blend = min(1.0, (step % formation_change_time) / formation_transition) if in_transition else 1.0
    
    # Update each drone
    for i, (drone_id, rotor_ids) in enumerate(drone_systems):
        # Get current position and orientation
        drone_pos, drone_orn = p.getBasePositionAndOrientation(drone_id)
        drone_vel, drone_ang_vel = p.getBaseVelocity(drone_id)
        
        # Calculate target position based on current and possibly previous formation
        current_target = formations[current_formation](step/100, i, num_drones)
        
        # If in transition, blend with previous formation
        if in_transition and step > formation_transition:
            prev_target = formations[prev_formation](step/100, i, num_drones)
            target_pos = np.array(prev_target[:3]) * (1-blend) + np.array(current_target[:3]) * blend
            
            # Blend orientation (needs special handling for angles)
            angle1, angle2 = prev_target[3], current_target[3]
            # Ensure we take the shortest path around the circle
            if abs(angle2 - angle1) > np.pi:
                if angle1 < angle2:
                    angle1 += 2 * np.pi
                else:
                    angle2 += 2 * np.pi
            target_angle = angle1 * (1-blend) + angle2 * blend
            target_pos = np.append(target_pos, target_angle)
        else:
            target_pos = np.array(current_target)
        
        # Store target position
        target_positions[i] = target_pos
        
        # Calculate PD control for position
        position_error = target_pos[:3] - np.array(drone_pos)
        velocity_error = -np.array(drone_vel)  # Target velocity is zero
        
        # PD control law
        force = kp * position_error + kd * velocity_error
        
        # Add gravity compensation
        force[2] += 9.8 * 0.5
        
        # Apply forces to main drone body
        p.applyExternalForce(
            drone_id,
            -1,  # Link ID (-1 means base)
            force,
            [0, 0, 0],  # Position (doesn't matter for -1)
            p.WORLD_FRAME
        )
        
        # Set orientation based on target
        target_orn = p.getQuaternionFromEuler([0, 0, target_pos[3]])
        current_euler = p.getEulerFromQuaternion(drone_orn)
        
        # Apply torque to align with target orientation
        orientation_error = np.array([0, 0, target_pos[3]]) - np.array([current_euler[0], current_euler[1], current_euler[2]])
        torque = np.array([0, 0, orientation_error[2] * 5.0])  # Simple P controller for yaw
        
        p.applyExternalTorque(
            drone_id,
            -1,
            torque,
            p.WORLD_FRAME
        )
        
        # Update rotor positions and rotations
        arm_length = 0.15
        arm_height = 0.01
        rotor_positions = [
            [arm_length, 0, arm_height],
            [0, arm_length, arm_height],
            [-arm_length, 0, arm_height],
            [0, -arm_length, arm_height]
        ]
        
        # Update rotor angles (alternating directions)
        rotor_directions = [1, -1, 1, -1]  # Alternate spinning directions for stability
        dt = 1/240
        
        for j, rotor_id in enumerate(rotor_ids):
            # Transform position based on drone orientation
            rel_pos = rotor_positions[j]
            matrix = p.getMatrixFromQuaternion(drone_orn)
            transformed_pos = [
                drone_pos[0] + matrix[0] * rel_pos[0] + matrix[1] * rel_pos[1] + matrix[2] * rel_pos[2],
                drone_pos[1] + matrix[3] * rel_pos[0] + matrix[4] * rel_pos[1] + matrix[5] * rel_pos[2],
                drone_pos[2] + matrix[6] * rel_pos[0] + matrix[7] * rel_pos[1] + matrix[8] * rel_pos[2]
            ]
            
            # Update rotor angle
            rotor_angles[i, j] += rotor_speed * rotor_directions[j] * dt
            rotor_angles[i, j] %= (2 * np.pi)
            
            # Apply rotor rotation
            rotor_orn = p.getQuaternionFromEuler([0, 0, rotor_angles[i, j]])
            final_orn = p.multiplyTransforms([0, 0, 0], drone_orn, [0, 0, 0], rotor_orn)[1]
            
            # Reset position and orientation of rotor
            p.resetBasePositionAndOrientation(rotor_id, transformed_pos, final_orn)
    
    # Step the simulation
    p.stepSimulation()
    
    # Slow down simulation for visibility
    time.sleep(1/240)

print("Simulation complete.")
p.disconnect()

pybullet build time: Apr 18 2025 03:31:46


Version = 4.1 Metal - 89.3
Vendor = Apple
Renderer = Apple M1
Connected with client ID: 0
Search path set to: /opt/anaconda3/envs/tf_310/lib/python3.10/site-packages/pybullet_data
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


2025-04-18 09:11:52.179 python[90624:7748243] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-18 09:11:52.179 python[90624:7748243] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [ ]:
#With reward system

import pybullet as p
import pybullet_data
import time
import numpy as np
import os

# Disconnect any existing connections
try:
    if p.isConnected():
        p.disconnect()
except:
    pass

# Try to connect with GUI
client = p.connect(p.GUI)
print(f"Connected with client ID: {client}")

# Set the path to PyBullet's data
p.setAdditionalSearchPath(pybullet_data.getDataPath())
print(f"Search path set to: {pybullet_data.getDataPath()}")

# Load floor plane
plane_id = p.loadURDF("plane.urdf")

# Set gravity
p.setGravity(0, 0, -9.8)

# Configure camera view
p.resetDebugVisualizerCamera(
    cameraDistance=10,
    cameraYaw=45,
    cameraPitch=-30,
    cameraTargetPosition=[0, 0, 2]
)

# Add a debug parameter to control the base height for all drones
height_slider = p.addUserDebugParameter("Base Height", 0.5, 5.0, 1.5)

# Number of drones
num_drones = 10

# Function to create a drone with rotors
def create_drone(position, color=None):
    if color is None:
        color = [0.7, 0.7, 0.7, 1]
    
    # Create the main body of the drone
    body_radius = 0.1
    body_height = 0.05
    body_visual = p.createVisualShape(
        shapeType=p.GEOM_CYLINDER,
        radius=body_radius,
        length=body_height,
        rgbaColor=color
    )
    body_collision = p.createCollisionShape(
        shapeType=p.GEOM_CYLINDER,
        radius=body_radius,
        height=body_height
    )
    
    # Create the drone base body
    drone_id = p.createMultiBody(
        baseMass=0.3,
        baseCollisionShapeIndex=body_collision,
        baseVisualShapeIndex=body_visual,
        basePosition=position,
        baseOrientation=p.getQuaternionFromEuler([0, 0, 0])
    )
    
    # Add four arms
    arm_length = 0.15
    arm_width = 0.02
    arm_height = 0.01
    arm_positions = [
        [arm_length/2, 0, 0],
        [0, arm_length/2, 0],
        [-arm_length/2, 0, 0],
        [0, -arm_length/2, 0]
    ]
    arm_orientations = [
        p.getQuaternionFromEuler([0, 0, 0]),
        p.getQuaternionFromEuler([0, 0, np.pi/2]),
        p.getQuaternionFromEuler([0, 0, 0]),
        p.getQuaternionFromEuler([0, 0, np.pi/2])
    ]
    
    arm_ids = []
    for i, (arm_pos, arm_orn) in enumerate(zip(arm_positions, arm_orientations)):
        arm_visual = p.createVisualShape(
            shapeType=p.GEOM_BOX,
            halfExtents=[arm_length/2, arm_width/2, arm_height/2],
            rgbaColor=color
        )
        arm_collision = p.createCollisionShape(
            shapeType=p.GEOM_BOX,
            halfExtents=[arm_length/2, arm_width/2, arm_height/2]
        )
        
        arm_id = p.createMultiBody(
            baseMass=0.05,
            baseCollisionShapeIndex=arm_collision,
            baseVisualShapeIndex=arm_visual,
            basePosition=[position[0] + arm_pos[0], position[1] + arm_pos[1], position[2] + arm_pos[2]],
            baseOrientation=arm_orn,
            baseInertialFramePosition=[0, 0, 0],
            baseInertialFrameOrientation=[0, 0, 0, 1],
            linkMasses=[],
            linkCollisionShapeIndices=[],
            linkVisualShapeIndices=[],
            linkPositions=[],
            linkOrientations=[],
            linkInertialFramePositions=[],
            linkInertialFrameOrientations=[],
            linkParentIndices=[],
            linkJointTypes=[],
            linkJointAxis=[]
        )
        arm_ids.append(arm_id)
    
    # Add rotors at the end of each arm
    rotor_radius = 0.05
    rotor_height = 0.01
    rotor_positions = [
        [arm_length, 0, arm_height],
        [0, arm_length, arm_height],
        [-arm_length, 0, arm_height],
        [0, -arm_length, arm_height]
    ]
    
    rotor_ids = []
    # Create different colored rotors for visual distinction between front/back/sides
    rotor_colors = [
        [1, 0, 0, 0.8],  # Red - front
        [0, 1, 0, 0.8],  # Green - right
        [0, 0, 1, 0.8],  # Blue - back
        [1, 1, 0, 0.8]   # Yellow - left
    ]
    
    for i, rotor_pos in enumerate(rotor_positions):
        # Create the rotor disk
        rotor_visual = p.createVisualShape(
            shapeType=p.GEOM_CYLINDER,
            radius=rotor_radius,
            length=rotor_height,
            rgbaColor=rotor_colors[i]
        )
        rotor_collision = p.createCollisionShape(
            shapeType=p.GEOM_CYLINDER,
            radius=rotor_radius,
            height=rotor_height
        )
        
        rotor_id = p.createMultiBody(
            baseMass=0.02,
            baseCollisionShapeIndex=rotor_collision,
            baseVisualShapeIndex=rotor_visual,
            basePosition=[position[0] + rotor_pos[0], position[1] + rotor_pos[1], position[2] + rotor_pos[2]],
            baseOrientation=p.getQuaternionFromEuler([0, 0, 0])
        )
        rotor_ids.append(rotor_id)
        
        # Add blades for visual effect (two crossed rods)
        blade_length = rotor_radius * 1.8
        blade_width = 0.005
        blade_height = 0.002
        
        # First blade
        blade_visual = p.createVisualShape(
            shapeType=p.GEOM_BOX,
            halfExtents=[blade_length/2, blade_width/2, blade_height/2],
            rgbaColor=[0.2, 0.2, 0.2, 0.9]
        )
        
        blade_id = p.createMultiBody(
            baseMass=0.001,  # Very light
            baseVisualShapeIndex=blade_visual,
            basePosition=[position[0] + rotor_pos[0], position[1] + rotor_pos[1], position[2] + rotor_pos[2] + rotor_height/2 + blade_height/2],
            baseOrientation=p.getQuaternionFromEuler([0, 0, 0])
        )
        rotor_ids.append(blade_id)
        
        # Second blade (perpendicular)
        blade_id2 = p.createMultiBody(
            baseMass=0.001,  # Very light
            baseVisualShapeIndex=blade_visual,
            basePosition=[position[0] + rotor_pos[0], position[1] + rotor_pos[1], position[2] + rotor_pos[2] + rotor_height/2 + blade_height/2],
            baseOrientation=p.getQuaternionFromEuler([0, 0, np.pi/2])
        )
        rotor_ids.append(blade_id2)
    
    return drone_id, arm_ids, rotor_ids

# Various formation patterns
def circle_formation(t, i, num_drones, base_height):
    radius = 3 + np.sin(t/2) * 1.5  # Dynamic radius
    angle = 2 * np.pi * i / num_drones + t / 5  # Rotating circle
    x = radius * np.cos(angle)
    y = radius * np.sin(angle)
    z = base_height + np.sin(t/3 + i * 0.5) * 0.5  # Vary height around base_height
    # Orientation: face the direction of travel
    orientation = np.arctan2(-np.sin(angle + t/5), -np.cos(angle + t/5))
    return [x, y, z, orientation]

def figure_eight_formation(t, i, num_drones, base_height):
    # Phase difference between drones
    phase = i * 2 * np.pi / num_drones
    # Figure-8 parametric equation
    x = 4 * np.sin(t/2 + phase)
    y = 4 * np.sin(t/2 + phase) * np.cos(t/2 + phase)
    z = base_height + np.sin(t + phase) * 0.5
    # Calculate direction of travel for orientation
    dx = 4 * np.cos(t/2 + phase)/2
    dy = 4 * (np.cos(t/2 + phase) * np.cos(t/2 + phase) - np.sin(t/2 + phase) * np.sin(t/2 + phase))/2
    orientation = np.arctan2(dy, dx)
    return [x, y, z, orientation]

def spiral_formation(t, i, num_drones, base_height):
    # Spiral with increasing radius
    angle = 4 * t + i * 2 * np.pi / num_drones
    radius = 1 + (i / num_drones) * 3
    x = radius * np.cos(angle)
    y = radius * np.sin(angle)
    z = base_height + (i / num_drones) * 1.5  # Use base_height as reference
    # Face the center of the spiral
    orientation = np.arctan2(-y, -x)
    return [x, y, z, orientation]

def wave_formation(t, i, num_drones, base_height):
    # Line with sine wave
    x = (i - num_drones/2) * 0.8
    y = np.sin(t + i * 0.5) * 2
    z = base_height + np.cos(t/2 + i * 0.3) * 0.7
    # Face the direction of travel
    dy = np.cos(t + i * 0.5) * 2
    orientation = np.arctan2(dy, 0)
    return [x, y, z, orientation]

def vertical_wave_formation(t, i, num_drones, base_height):
    # Drones arranged in a circle but with prominent vertical waves
    angle = 2 * np.pi * i / num_drones
    radius = 3
    x = radius * np.cos(angle)
    y = radius * np.sin(angle)
    # More pronounced vertical movement
    z = base_height + np.sin(t + i * 2 * np.pi / num_drones) * 1.5
    orientation = np.arctan2(-np.sin(angle), -np.cos(angle))
    return [x, y, z, orientation]

# List of formations
formations = [
    circle_formation,
    figure_eight_formation,
    spiral_formation,
    wave_formation,
    vertical_wave_formation
]

# ---- NEW: REWARD SYSTEM COMPONENTS ----

# Reward parameters and weights
REWARD_WEIGHTS = {
    'position_tracking': 1.0,    # How well drones follow their target positions
    'orientation_tracking': 0.5, # How well drones maintain target orientation
    'energy_efficiency': 0.3,    # Rewards lower energy consumption
    'smoothness': 0.7,           # Rewards smooth movements
    'collision_avoidance': 2.0,  # High penalty for getting too close to other drones
    'formation_cohesion': 1.0    # Rewards staying in formation
}

# Create reward text displays for each drone
reward_text_ids = [-1] * num_drones
cumulative_rewards = np.zeros(num_drones)
instantaneous_rewards = np.zeros(num_drones)

# Collision detection parameters
min_safe_distance = 0.5  # Minimum safe distance between drones

# Add reward mode slider (different reward emphasis)
reward_mode_slider = p.addUserDebugParameter("Reward Mode", 0, 4, 0)
reward_modes = [
    "Balanced",
    "Tracking Focus", 
    "Energy Focus",
    "Safety Focus",
    "Formation Focus"
]

# Create text display for reward mode
reward_mode_text_id = p.addUserDebugText(
    f"Reward Mode: {reward_modes[0]}", 
    [0, -4, 4],
    textColorRGB=[1, 1, 0],
    textSize=1.5
)

# Function to calculate rewards based on drone performance
def calculate_rewards(drone_systems, target_positions, prev_positions, prev_velocities, step):
    rewards = np.zeros(len(drone_systems))
    
    # Get current reward mode
    mode_index = int(p.readUserDebugParameter(reward_mode_slider))
    
    # Adjust weights based on mode
    weights = REWARD_WEIGHTS.copy()
    if mode_index == 1:  # Tracking Focus
        weights['position_tracking'] *= 2.0
        weights['orientation_tracking'] *= 2.0
    elif mode_index == 2:  # Energy Focus
        weights['energy_efficiency'] *= 3.0
    elif mode_index == 3:  # Safety Focus
        weights['collision_avoidance'] *= 3.0
    elif mode_index == 4:  # Formation Focus
        weights['formation_cohesion'] *= 3.0
    
    # Update reward mode display
    p.removeUserDebugItem(reward_mode_text_id)
    mode_text_id = p.addUserDebugText(
        f"Reward Mode: {reward_modes[mode_index]}", 
        [0, -4, 4],
        textColorRGB=[1, 1, 0],
        textSize=1.5,
        replaceItemUniqueId=reward_mode_text_id
    )
    
    # Calculate drone positions for collision detection
    drone_positions = []
    for drone_id, _, _ in drone_systems:
        pos, _ = p.getBasePositionAndOrientation(drone_id)
        drone_positions.append(np.array(pos))
    
    # Process each drone
    for i, (drone_id, _, _) in enumerate(drone_systems):
        reward = 0.0
        
        # Get current state
        pos, orn = p.getBasePositionAndOrientation(drone_id)
        vel, ang_vel = p.getBaseVelocity(drone_id)
        
        # Position tracking reward: negative distance to target
        target_pos = target_positions[i][:3]
        position_error = np.linalg.norm(np.array(pos) - np.array(target_pos))
        position_reward = np.exp(-2.0 * position_error)  # Exponential decay
        reward += weights['position_tracking'] * position_reward
        
        # Orientation tracking
        target_angle = target_positions[i][3]
        current_euler = p.getEulerFromQuaternion(orn)
        angle_error = min(abs(target_angle - current_euler[2]), 
                          abs(target_angle - current_euler[2] + 2*np.pi),
                          abs(target_angle - current_euler[2] - 2*np.pi))
        orientation_reward = np.exp(-3.0 * angle_error)
        reward += weights['orientation_tracking'] * orientation_reward
        
        # Energy efficiency: penalize excessive forces/movements
        velocity_magnitude = np.linalg.norm(vel)
        energy_usage = velocity_magnitude ** 2  # Proportional to kinetic energy
        energy_reward = np.exp(-0.3 * energy_usage)
        reward += weights['energy_efficiency'] * energy_reward
        
        # Movement smoothness (avoid jerky movements)
        if step > 0 and prev_positions[i] is not None and prev_velocities[i] is not None:
            acceleration = np.linalg.norm(np.array(vel) - np.array(prev_velocities[i]))
            smoothness_reward = np.exp(-0.5 * acceleration)
            reward += weights['smoothness'] * smoothness_reward
        
        # Collision avoidance
        collision_penalty = 0
        for j, other_pos in enumerate(drone_positions):
            if i != j:  # Don't check against self
                distance = np.linalg.norm(np.array(pos) - other_pos)
                if distance < min_safe_distance:
                    # Penalty increases as drones get closer
                    collision_penalty += ((min_safe_distance - distance) / min_safe_distance) * 2.0
        
        reward -= weights['collision_avoidance'] * collision_penalty
        
        # Formation cohesion (how well the drone maintains its relative position in formation)
        # For this, we'll calculate how well the drone maintains its position relative to the centroid
        centroid = np.mean(drone_positions, axis=0)
        ideal_vector = np.array(target_pos) - centroid
        actual_vector = np.array(pos) - centroid
        
        vector_diff = np.linalg.norm(ideal_vector - actual_vector)
        formation_reward = np.exp(-vector_diff)
        reward += weights['formation_cohesion'] * formation_reward
        
        # Store the reward
        rewards[i] = reward
    
    return rewards

# Create visual targets for each drone (small spheres at target positions)
target_visual_ids = [-1] * num_drones

# Add a debug parameter to show/hide targets
show_targets_param = p.addUserDebugParameter("Show Targets", 0, 1, 1)

# ---- END NEW REWARD SYSTEM COMPONENTS ----

# Add keyboard controls for manual height adjustment
height_offset = 0.0
height_increment = 0.1
p.addUserDebugText("Press 'U' to move up, 'D' to move down", [0, 0, 0], [1, 1, 1])

# Run simulation
print("Running simulation with dynamic drone movements and reward system...")
print("Press 'U' to increase height, 'D' to decrease height")
print("Use the sliders to set base height and reward mode")

current_formation = 0
formation_change_time = 500  # Steps before changing formation
formation_transition = 100   # Steps for smooth transition

# Improved PD controller parameters for smoother movement
kp = 15.0  # Position gain (increased for better tracking)
kd = 8.0   # Velocity gain (increased for better damping)
kp_z = 20.0  # Special Z position gain (increased for better height control)
kd_z = 10.0  # Special Z velocity gain

# Target positions and velocities
target_positions = [np.zeros(4) for _ in range(num_drones)]  # x, y, z, orientation
current_velocities = [np.zeros(3) for _ in range(num_drones)]

# Store previous positions and velocities for reward calculation
prev_positions = [None] * num_drones
prev_velocities = [None] * num_drones

# Rotor animation speed
rotor_speed = 30.0  # radians per second
rotor_angles = np.zeros((num_drones, 4))  # Track rotation angles for each rotor

# Add debug visualization lines for drone trajectory
line_ids = [-1] * num_drones
debug_lines = [[] for _ in range(num_drones)]
max_debug_points = 100  # Maximum points in the trajectory line

# Create drones with all their components
drone_systems = []
for i in range(num_drones):
    # Calculate position in a circle
    angle = 2 * np.pi * i / num_drones
    x = 3 * np.cos(angle)
    y = 3 * np.sin(angle)
    z = 1.5  # Start at same height
    
    # Create a colored drone
    color = [i/num_drones, 0.5, 1-i/num_drones, 1]  # Different color for each drone
    drone_id, arm_ids, rotor_ids = create_drone([x, y, z], color)
    drone_systems.append((drone_id, arm_ids, rotor_ids))

# Create leaderboard text display
leaderboard_text_id = -1

# Variable to track whether to calculate rewards this step
calculate_rewards_this_step = False

for step in range(6000):
    # Process keyboard events for height control
    keys = p.getKeyboardEvents()
    for key, state in keys.items():
        if state & p.KEY_WAS_TRIGGERED:
            if key == ord('u') or key == ord('U'):
                height_offset += height_increment
                print(f"Height offset: {height_offset:.2f}")
            elif key == ord('d') or key == ord('D'):
                height_offset -= height_increment
                print(f"Height offset: {height_offset:.2f}")
    
    # Get base height from slider
    base_height = p.readUserDebugParameter(height_slider) + height_offset
    
    # Change formation periodically
    if step % formation_change_time == 0:
        prev_formation = current_formation
        current_formation = (current_formation + 1) % len(formations)
        print(f"Changing to formation: {formations[current_formation].__name__}")
    
    # Calculate blend factor for smooth transition
    in_transition = step % formation_change_time < formation_transition
    blend = min(1.0, (step % formation_change_time) / formation_transition) if in_transition else 1.0
    
    # Determine if rewards should be calculated this step (every 10 steps)
    calculate_rewards_this_step = (step % 10 == 0)
    
    # Check if targets should be shown
    show_targets = p.readUserDebugParameter(show_targets_param) > 0.5
    
    # Update each drone
    for i, (drone_id, arm_ids, rotor_ids) in enumerate(drone_systems):
        # Get current position and orientation
        drone_pos, drone_orn = p.getBasePositionAndOrientation(drone_id)
        drone_vel, drone_ang_vel = p.getBaseVelocity(drone_id)
        
        # Store previous position and velocity for reward calculation
        prev_positions[i] = np.array(drone_pos)
        prev_velocities[i] = np.array(drone_vel)
        
        # Calculate target position based on current and possibly previous formation
        current_target = formations[current_formation](step/100, i, num_drones, base_height)
        
        # If in transition, blend with previous formation
        if in_transition and step > formation_transition:
            prev_target = formations[prev_formation](step/100, i, num_drones, base_height)
            target_pos = np.array(prev_target[:3]) * (1-blend) + np.array(current_target[:3]) * blend
            
            # Blend orientation (needs special handling for angles)
            angle1, angle2 = prev_target[3], current_target[3]
            # Ensure we take the shortest path around the circle
            if abs(angle2 - angle1) > np.pi:
                if angle1 < angle2:
                    angle1 += 2 * np.pi
                else:
                    angle2 += 2 * np.pi
            target_angle = angle1 * (1-blend) + angle2 * blend
            target_pos = np.append(target_pos, target_angle)
        else:
            target_pos = np.array(current_target)
        
        # Store target position
        target_positions[i] = target_pos
        
        # Show or hide target markers
        if show_targets:
            # Update target visualization
            target_visual_color = [i/num_drones, 0.5, 1-i/num_drones, 0.3]  # Transparent version of drone color
            if target_visual_ids[i] == -1:
                # Create target visualization if it doesn't exist
                target_visual_ids[i] = p.createVisualShape(
                    shapeType=p.GEOM_SPHERE,
                    radius=0.1,
                    rgbaColor=target_visual_color
                )
                p.createMultiBody(
                    baseMass=0,  # Static object
                    baseVisualShapeIndex=target_visual_ids[i],
                    basePosition=target_pos[:3]
                )
            else:
                # Update position of existing target
                p.resetBasePositionAndOrientation(
                    target_visual_ids[i],
                    target_pos[:3],
                    [0, 0, 0, 1]  # Identity quaternion
                )
        elif target_visual_ids[i] != -1:
            # Hide target
            p.removeBody(target_visual_ids[i])
            target_visual_ids[i] = -1
        
        # Calculate PD control for position
        position_error = target_pos[:3] - np.array(drone_pos)
        velocity_error = -np.array(drone_vel)  # Target velocity is zero
        
        # PD control law with separate gains for Z axis for better stability
        force = np.array([
            kp * position_error[0] + kd * velocity_error[0],
            kp * position_error[1] + kd * velocity_error[1],
            kp_z * position_error[2] + kd_z * velocity_error[2]
        ])
        
        # Add gravity compensation
        force[2] += 9.8 * 0.5
        
        # Apply dynamic thrust based on orientation (more realistic)
        # When drone is tilted, it generates less upward force
        euler = p.getEulerFromQuaternion(drone_orn)
        tilt_factor = np.cos(euler[0]) * np.cos(euler[1])
        force[2] /= max(0.5, tilt_factor)  # Increase thrust when tilted
        
        # Apply forces to main drone body
        p.applyExternalForce(
            drone_id,
            -1,  # Link ID (-1 means base)
            force,
            [0, 0, 0],  # Position (doesn't matter for -1)
            p.WORLD_FRAME
        )
        
        # Set orientation based on target
        target_orn = p.getQuaternionFromEuler([0, 0, target_pos[3]])
        current_euler = p.getEulerFromQuaternion(drone_orn)
        
        # Apply torque to align with target orientation
        orientation_error = np.array([0, 0, target_pos[3]]) - np.array([current_euler[0], current_euler[1], current_euler[2]])
        
        # Add angular velocity damping for stability
        ang_damping = -np.array(drone_ang_vel) * 0.5
        torque = np.array([
            ang_damping[0],
            ang_damping[1],
            orientation_error[2] * 5.0 + ang_damping[2]
        ])
        
        p.applyExternalTorque(
            drone_id,
            -1,
            torque,
            p.WORLD_FRAME
        )
        
        # Update arm positions
        arm_length = 0.15
        arm_height = 0.01
        arm_positions = [
            [arm_length/2, 0, 0],
            [0, arm_length/2, 0],
            [-arm_length/2, 0, 0],
            [0, -arm_length/2, 0]
        ]
        
        for j, arm_id in enumerate(arm_ids):
            # Transform position based on drone orientation
            rel_pos = arm_positions[j]
            matrix = p.getMatrixFromQuaternion(drone_orn)
            transformed_pos = [
                drone_pos[0] + matrix[0] * rel_pos[0] + matrix[1] * rel_pos[1] + matrix[2] * rel_pos[2],
                drone_pos[1] + matrix[3] * rel_pos[0] + matrix[4] * rel_pos[1] + matrix[5] * rel_pos[2],
                drone_pos[2] + matrix[6] * rel_pos[0] + matrix[7] * rel_pos[1] + matrix[8] * rel_pos[2]
            ]
            
            # Arm inherits orientation from drone body
            p.resetBasePositionAndOrientation(arm_id, transformed_pos, drone_orn)
        
        # Update rotor positions and rotations
        rotor_positions = [
            [arm_length, 0, arm_height],
            [0, arm_length, arm_height],
            [-arm_length, 0, arm_height],
            [0, -arm_length, arm_height]
        ]
        
        # Update rotor angles (alternating directions)
        rotor_directions = [1, -1, 1, -1]  # Alternate spinning directions for stability
        dt = 1/240
        
        # Calculate rotor speeds proportionally to the forces needed
        # Higher when accelerating/decelerating
        force_magnitude = np.linalg.norm(force)
        rotor_speed_factor = 1.0 + 0.5 * min(1.0, force_magnitude / 20.0)
        
        # Three rotors per position (disk + 2 blades)
        for j in range(0, len(rotor_ids), 3):
            # Get base index for this rotor set
            base_j = j // 3
            
            # Transform position based on drone orientation
            rel_pos = rotor_positions[base_j]
            matrix = p.getMatrixFromQuaternion(drone_orn)
            transformed_pos = [
                drone_pos[0] + matrix[0] * rel_pos[0] + matrix[1] * rel_pos[1] + matrix[2] * rel_pos[2],
                drone_pos[1] + matrix[3] * rel_pos[0] + matrix[4] * rel_pos[1] + matrix[5] * rel_pos[2],
                drone_pos[2] + matrix[6] * rel_pos[0] + matrix[7] * rel_pos[1] + matrix[8] * rel_pos[2]
            ]
            
            # Update rotor angle
            rotor_angles[i, base_j] += rotor_speed * rotor_speed_factor * rotor_directions[base_j] * dt
            rotor_angles[i, base_j] %= (2 * np.pi)
            
            # Reset position and orientation of rotor disk
            rotor_orn = p.getQuaternionFromEuler([0, 0, 0])
            final_orn = p.multiplyTransforms([0, 0, 0], drone_orn, [0, 0, 0], rotor_orn)[1]
            p.resetBasePositionAndOrientation(rotor_ids[j], transformed_pos, final_orn)
            
            # Rotate blades
            blade_orn1 = p.getQuaternionFromEuler([0, 0, rotor_angles[i, base_j]])
            blade_orn2 = p.getQuaternionFromEuler([0, 0, rotor_angles[i, base_j] + np.pi/2])
            
            final_blade_orn1 = p.multiplyTransforms([0, 0, 0], drone_orn, [0, 0, 0], blade_orn1)[1]
            final_blade_orn2 = p.multiplyTransforms([0, 0, 0], drone_orn, [0, 0, 0], blade_orn2)[1]
            
            # Position for blades is slightly above the rotor disk
            blade_pos = [
                transformed_pos[0],
                transformed_pos[1],
                transformed_pos[2] + 0.01  # Slightly above the rotor disk
            ]
            
            p.resetBasePositionAndOrientation(rotor_ids[j+1], blade_pos, final_blade_orn1)
            p.resetBasePositionAndOrientation(rotor_ids[j+2], blade_pos, final_blade_orn2)
        
        # Add visualization of drone trajectory
        if step % 10 == 0:  # Update every 10 steps to save performance
            debug_lines[i].append(drone_pos)
            if len(debug_lines[i]) > max_debug_points:
                debug_lines[i].pop(0)
            
            if len(debug_lines[i]) > 1:
                # Remove previous line
                if line_ids[i] >= 0:
                    p.removeUserDebugItem(line_ids[i])
                
                # Create line points
                points = []
                for point in debug_lines[i]:
                    points.extend(point)
                
                # Draw new line with color matching the drone
                line_ids[i] = p.addUserDebugLine(
                    debug_lines[i][0],
                    debug_lines[i][-1],
                    lineColorRGB=[i/num_drones, 0.5, 1-i/num_drones],
                    lineWidth=2,
                    lifeTime=0,
                    replaceItemUniqueId=line_ids[i]
                )
    
    # Calculate rewards every 10 steps to save computation
    if calculate_rewards_this_step:
        # Calculate rewards for each drone
        rewards = calculate_rewards(
            drone_systems, 
            target_positions, 
            prev_positions, 
            prev_velocities, 
            step
        )
        
        # Update instantaneous rewards
        instantaneous_rewards = rewards
        
        # Update cumulative rewards
        cumulative_rewards += rewards
        
        # Update reward text displays
        for i in range(num_drones):
            # Remove previous text
            if reward_text_ids[i] != -1:
                p.removeUserDebugItem(reward_text_ids[i])
            
            # Get drone position
            drone_pos, _ = p.getBasePositionAndOrientation(drone_systems[i][0])
            
            # Create text with current reward
            text_pos = [drone_pos[0], drone_pos[1], drone_pos[2] + 0.3]
            reward_text = f"{instantaneous_rewards[i]:.2f}"
            
            # Color based on reward (green for good, red for bad)
            if instantaneous_rewards[i] >= 2.0:
                text_color = [0, 1, 0]  # Green
            elif instantaneous_rewards[i] >= 1.0:
                text_color = [1, 1, 0]  # Yellow
            else:
                text_color = [1, 0, 0]  # Red
            
            # Add text to display
            reward_text_ids[i] = p.addUserDebugText(
                reward_text,
                text_pos,
                textColorRGB=text_color,
                textSize=1.0
            )
        
        # Update leaderboard display
        if leaderboard_text_id != -1:
            p.removeUserDebugItem(leaderboard_text_id)
        
        # Sort drones by cumulative reward
        sorted_indices = np.argsort(cumulative_rewards)[::-1]  # Descending order
        
        # Create leaderboard text
        leaderboard_text = "Leaderboard (Cumulative Rewards):\n"
        for rank, idx in enumerate(sorted_indices[:5]):  # Show top 5
            leaderboard_text += f"{rank+1}. Drone {idx}: {cumulative_rewards[idx]:.2f}\n"
        
        # Add text at a fixed position
        leaderboard_text_id = p.addUserDebugText(
            leaderboard_text,
            [5, 5, 5],  # Position in 3D space
            textColorRGB=[1, 1, 1],
            textSize=1.0
        )
    
    # Step the simulation
    p.stepSimulation()
    
    # Slow down simulation for visibility
    time.sleep(1/240)

print("Simulation complete.")
p.disconnect()





pybullet build time: Apr 18 2025 03:31:46


In [ ]:
'''# Try to connect with GUI
try:
    client = p.connect(p.GUI)
    print(f"Connected with client ID: {client}")
except:
    # Fall back to DIRECT mode if GUI fails
    client = p.connect(p.DIRECT)
    print(f"GUI connection failed, using DIRECT mode with client ID: {client}")

# Set the path to PyBullet's data
p.setAdditionalSearchPath(pybullet_data.getDataPath())
print(f"Search path set to: {pybullet_data.getDataPath()}")

# Add a debug parameter to control the base height for all drones
try:
    height_slider = p.addUserDebugParameter("Base Height", 0.5, 5.0, 1.5)
    print(f"Height slider created with ID: {height_slider}")
except Exception as e:
    print(f"Failed to create height slider: {e}")
    # Provide a default value
    height_slider = None'''